In [1]:
import zipfile
import pandas as pd
from association_alarm_id_to_lp_id import associate_alarm_id_to_lp_id

## Unzip Alarms Datasets

In [3]:
# Percorso del file ZIP da estrarre
zip_mob_file_path = "alarms datasets/mob/20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.zip"
zip_tx_adsl_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.zip"
zip_tx_pdh_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.zip"
zip_tx_ptn_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.zip"
zip_tx_sdh_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.zip"

# Percorso della cartella di destinazione per l'estrazione
extract_to_folder_mob = "alarms datasets/mob"
extract_to_folder_tx = "alarms datasets/tx"

# Percorso del file ZIP da estrarre
zip_files_paths = {
    zip_mob_file_path: extract_to_folder_mob,
    zip_tx_adsl_file_path: extract_to_folder_tx,
    zip_tx_pdh_file_path: extract_to_folder_tx,
    zip_tx_ptn_file_path: extract_to_folder_tx,
    zip_tx_sdh_file_path: extract_to_folder_tx
}

# Aprire il file ZIP
for zip_file_path, extract_to_folder in zip_files_paths.items():
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Estrarre tutto il contenuto nella cartella di destinazione
        zip_ref.extractall(extract_to_folder)

## Load Datasets in a Dataframe

In [2]:
mob_alarms_df = pd.read_parquet('alarms datasets/mob/20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

In [3]:
tx_alarms_adsl_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')
tx_alarms_pdh_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')
tx_alarms_ptn_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')
tx_alarms_sdh_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet')

## Clean Alarms Datasets removing old "lavori programmati" columns

In [3]:
filtered_mob_alarms_df = mob_alarms_df.drop(columns=['lp_type', 'lp_id'])

In [4]:
filtered_tx_alarms_adsl_df = tx_alarms_adsl_df.drop(columns=['lp_type', 'lp_id'])

In [5]:
filtered_tx_alarms_pdh_df = tx_alarms_pdh_df.drop(columns=['lp_type', 'lp_id'])

In [6]:
filtered_tx_alarms_ptn_df = tx_alarms_ptn_df.drop(columns=['lp_type', 'lp_id'])

In [7]:
filtered_tx_alarms_sdh_df = tx_alarms_sdh_df.drop(columns=['lp_type', 'lp_id'])

## Save Cleaned Datasets 


In [4]:
filtered_mob_alarms_df.to_parquet('alarms datasets/mob/[filtered]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

In [8]:
filtered_tx_alarms_adsl_df.to_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')

In [9]:
filtered_tx_alarms_pdh_df.to_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')

In [10]:
filtered_tx_alarms_ptn_df.to_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')

In [11]:
filtered_tx_alarms_sdh_df.to_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet')

## Associate Alarms with Lavori Programmati

In [14]:
lavori_programmati_df = pd.read_csv("lavori programmati/20230101_20240101_export_cac_all_preprocessed.csv")

In [ ]:
new_mob_alarms_df = associate_alarm_id_to_lp_id(filtered_mob_alarms_df, lavori_programmati_df)

In [15]:
new_tx_alarms_adsl_df = associate_alarm_id_to_lp_id(filtered_tx_alarms_adsl_df, lavori_programmati_df)

In [ ]:
new_tx_alarms_pdh_df = associate_alarm_id_to_lp_id(filtered_tx_alarms_pdh_df, lavori_programmati_df)

In [ ]:
new_tx_alarms_ptn_df = associate_alarm_id_to_lp_id(filtered_tx_alarms_ptn_df, lavori_programmati_df)

## Associate Alarms with Lavori Programmati v2

In [1]:
filtered_mob_alarms_df = pd.read_parquet('alarms datasets/mob/[filtered]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

NameError: name 'pd' is not defined

In [3]:
filtered_tx_alarms_adsl_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')

In [4]:
filtered_tx_alarms_pdh_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')

In [5]:
filtered_tx_alarms_ptn_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')

In [6]:
filtered_tx_alarms_sdh_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet')

In [6]:
lavori_programmati_short_df = pd.read_csv("lavori programmati/20230101_20240101_lp.csv")
lavori_programmati_short_df = lavori_programmati_short_df.iloc[:, 1:] # drop the first column

In [7]:
lavori_programmati_short_df

,alarm_id,lp_id,lp_type
0,COS1B;119303965,['3018.01/2022'],['CAVO']
1,COS1B;119303970,['3018.01/2022'],['CAVO']
2,COS1B;119303974,['3018.01/2022'],['CAVO']
3,COS1B;119303994,['3018/2022' '3018.01/2022'],['CAVO']
4,COS1B;119304000,['3018.01/2022'],['CAVO']
...,...,...,...
144034,EFMTX;6583a1f9469e8c000136ff65,['2465/2023'],['CAVO']
144035,EFMTX;6583a7fa469e8c0001370239,['2465/2023'],['CAVO']
144036,EFMTX;6583a7fa469e8c000137023b,['2465/2023'],['CAVO']
144037,EFMTX;6583b543ba43af0001748f94,['2465/2023'],['CAVO']


In [8]:
new_mob_alarms_df = filtered_mob_alarms_df.merge(lavori_programmati_short_df, on='alarm_id', how='left')
new_mob_alarms_df.to_parquet('alarms datasets/mob/[new]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

In [9]:
new_tx_alarms_adsl_df = filtered_tx_alarms_adsl_df.merge(lavori_programmati_short_df, on='alarm_id', how='left')
new_tx_alarms_adsl_df.to_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')

In [12]:
new_tx_alarms_pdh_df = filtered_tx_alarms_pdh_df.merge(lavori_programmati_short_df, on='alarm_id', how='left')
new_tx_alarms_pdh_df.to_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')

In [10]:
filtered_mob_alarms_df

,alarm_id,data,first_occurrence,first_severity,last_occurrence,last_update,loc_descr,make_prediction,ne_id,network,...,province_cat,is_note_present,is_ack_or_notes,is_tt,is_abam_tt,loc_name,tt_id,link_id,loc_name_a,loc_name_z
0,EFM;63b098b01a65d200011620f8,2023-01-01 00:00:02,2022-12-31 21:16:48,critical,2022-12-31 23:37:41,2022-12-31 23:58:25,<NA>,False,etr100009,an,...,NaN,False,False,False,False,NaN,<NA>,NaN,None,None
1,EFM;63b0b476c623b40001a68b45,2023-01-01 00:00:02,2022-12-31 23:15:18,major,2022-12-31 23:20:09,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,fo,False,False,False,False,ceseitej,<NA>,cxa2pc7,None,None
2,EFM;63b0b5a8c623b40001a68b9b,2023-01-01 00:00:02,2022-12-31 23:20:24,major,2022-12-31 23:20:49,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,fo,False,False,False,False,ceseitej,<NA>,cxa2pc7,None,None
3,EFM;63b0b5c6c623b40001a68ba2,2023-01-01 00:00:02,2022-12-31 23:20:54,major,2022-12-31 23:23:09,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,fo,False,False,False,False,ceseitej,<NA>,cxa2pc7,None,None
4,EFM;63b0b675c623b40001a68c03,2023-01-01 00:00:02,2022-12-31 23:23:49,major,2022-12-31 23:32:15,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,fo,False,False,False,False,ceseitej,<NA>,cxa2pc7,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28176439,EFM;6591f1f639628d000137853b,2024-01-01 00:00:03,2023-12-31 23:57:58,major,2023-12-31 23:59:42,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,NaN,False,False,False,False,NaN,<NA>,NaN,None,None
28176440,EFM;6591f1f739628d000137854e,2024-01-01 00:00:03,2023-12-31 23:57:58,major,2023-12-31 23:59:52,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,NaN,False,False,False,False,NaN,<NA>,NaN,None,None
28176441,EFM;6591f1f739628d0001378554,2024-01-01 00:00:03,2023-12-31 23:57:58,major,2023-12-31 23:59:52,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,NaN,False,False,False,False,NaN,<NA>,NaN,None,None
28176442,EFM;6591f1f939628d0001378585,2024-01-01 00:00:03,2023-12-31 23:57:59,major,2023-12-31 23:59:42,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,NaN,False,False,False,False,NaN,<NA>,NaN,None,None


In [11]:
new_mob_alarms_df

,alarm_id,data,first_occurrence,first_severity,last_occurrence,last_update,loc_descr,make_prediction,ne_id,network,...,is_ack_or_notes,is_tt,is_abam_tt,loc_name,tt_id,link_id,loc_name_a,loc_name_z,lp_id,lp_type
0,EFM;63b098b01a65d200011620f8,2023-01-01 00:00:02,2022-12-31 21:16:48,critical,2022-12-31 23:37:41,2022-12-31 23:58:25,<NA>,False,etr100009,an,...,False,False,False,NaN,<NA>,NaN,None,None,NaN,NaN
1,EFM;63b0b476c623b40001a68b45,2023-01-01 00:00:02,2022-12-31 23:15:18,major,2022-12-31 23:20:09,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,False,False,False,ceseitej,<NA>,cxa2pc7,None,None,NaN,NaN
2,EFM;63b0b5a8c623b40001a68b9b,2023-01-01 00:00:02,2022-12-31 23:20:24,major,2022-12-31 23:20:49,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,False,False,False,ceseitej,<NA>,cxa2pc7,None,None,NaN,NaN
3,EFM;63b0b5c6c623b40001a68ba2,2023-01-01 00:00:02,2022-12-31 23:20:54,major,2022-12-31 23:23:09,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,False,False,False,ceseitej,<NA>,cxa2pc7,None,None,NaN,NaN
4,EFM;63b0b675c623b40001a68c03,2023-01-01 00:00:02,2022-12-31 23:23:49,major,2022-12-31 23:32:15,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,False,False,False,ceseitej,<NA>,cxa2pc7,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28176439,EFM;6591f1f639628d000137853b,2024-01-01 00:00:03,2023-12-31 23:57:58,major,2023-12-31 23:59:42,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,False,False,False,NaN,<NA>,NaN,None,None,NaN,NaN
28176440,EFM;6591f1f739628d000137854e,2024-01-01 00:00:03,2023-12-31 23:57:58,major,2023-12-31 23:59:52,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,False,False,False,NaN,<NA>,NaN,None,None,NaN,NaN
28176441,EFM;6591f1f739628d0001378554,2024-01-01 00:00:03,2023-12-31 23:57:58,major,2023-12-31 23:59:52,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,False,False,False,NaN,<NA>,NaN,None,None,NaN,NaN
28176442,EFM;6591f1f939628d0001378585,2024-01-01 00:00:03,2023-12-31 23:57:59,major,2023-12-31 23:59:42,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,False,False,False,NaN,<NA>,NaN,None,None,NaN,NaN


In [13]:
new_tx_alarms_ptn_df = filtered_tx_alarms_ptn_df.merge(lavori_programmati_short_df, on='alarm_id', how='left')
new_tx_alarms_ptn_df.to_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')

In [14]:
new_tx_alarms_sdh_df = filtered_tx_alarms_sdh_df.merge(lavori_programmati_short_df, on='alarm_id', how='left')
new_tx_alarms_sdh_df.to_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet')